In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time
tqdm.tqdm.pandas()

In [2]:
bulk_df = pd.read_excel('./공공주택찾기/강원도/일괄계약전체_20180101_20220224.xlsx')
print(bulk_df.shape)
bulk_df.head()

(716, 15)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약
0,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012,2019-03-19,138.084633,10,연립다세대,Y
1,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012,2019-03-19,138.084633,10,연립다세대,Y
2,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012,2019-03-19,138.084633,10,연립다세대,Y
3,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,2,2012,2019-03-19,138.084633,10,연립다세대,Y
4,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012,2019-03-19,138.084633,10,연립다세대,Y


In [3]:
bulk_df['소유자'] = np.nan

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Get LATEST driver version for 98.0.4758
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\max6296\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [10]:
driver.get("http://www.iros.go.kr/PMainJ.jsp")

In [11]:
driver.get("http://www.iros.go.kr/frontservlet?cmd=RISUWelcomeViewC")

In [12]:
def get_owner_name(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        owner_name = np.nan        
        addr_df = df[df['지번주소'] == addr]
        addr = addr.replace('-0', '')
        
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element_by_xpath('html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        time.sleep(1)
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(1)
        
        elem_result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [13]:
prac_df = get_owner_name(bulk_df)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/26 [00:00<?, ?it/s]

In [14]:
print(prac_df.shape)
prac_df.head()

(716, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
0,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012,2019-03-19,138.084633,10,연립다세대,Y,주식회사더***~
1,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012,2019-03-19,138.084633,10,연립다세대,Y,주식회사더***~
2,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012,2019-03-19,138.084633,10,연립다세대,Y,주식회사더***~
3,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,2,2012,2019-03-19,138.084633,10,연립다세대,Y,주식회사더***~
4,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012,2019-03-19,138.084633,10,연립다세대,Y,주식회사더***~


In [15]:
prac_df['소유자'].value_counts()

한***~        222
정***~        130
신***~         49
주식회사다***~     43
주식회사라***~     36
이**           34
주**           34
정**외 1명       20
주**외 1명       19
김**           19
박**           18
정**           17
주식회사더***~     17
주식회사무***~     14
홍**           12
남**           12
최**           10
탁**           10
Name: 소유자, dtype: int64

In [16]:
prac_df['소유자'].isna().sum()

0

In [17]:
prac_df = prac_df.dropna(subset=['소유자']).reset_index(drop=True)

In [18]:
prac_df.shape

(716, 16)

In [19]:
def sh_lh(x):
    if x == '한***~':
        return 'LH'
    else:
        return x

In [20]:
prac_df['소유자'] = prac_df['소유자'].apply(sh_lh)

In [21]:
prac_df['소유자'].value_counts()

LH           222
정***~        130
신***~         49
주식회사다***~     43
주식회사라***~     36
이**           34
주**           34
정**외 1명       20
주**외 1명       19
김**           19
박**           18
정**           17
주식회사더***~     17
주식회사무***~     14
홍**           12
남**           12
최**           10
탁**           10
Name: 소유자, dtype: int64

In [22]:
%%time
prac_df.to_excel('./공공주택찾기/인천/일괄계약_전체_소유자포함_20180101_20220224.xlsx', index=False)

Wall time: 221 ms


In [23]:
prac_df = prac_df[
    (prac_df['소유자'] == 'LH')
]
print(prac_df.shape)
prac_df.head()

(222, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
159,강원도 강릉시 포남동 1169-0,성덕포남로168번안길 3,강원도 강릉시 포남동,1169,0,(1169-0),60.0215,15000,5,2021,2021-08-17,249.910449,1,연립다세대,Y,LH
160,강원도 강릉시 포남동 1169-0,성덕포남로168번안길 3,강원도 강릉시 포남동,1169,0,(1169-0),60.0215,15000,5,2021,2021-08-17,249.910449,1,연립다세대,Y,LH
161,강원도 강릉시 포남동 1169-0,성덕포남로168번안길 3,강원도 강릉시 포남동,1169,0,(1169-0),60.0215,15300,4,2021,2021-08-17,254.908658,1,연립다세대,Y,LH
162,강원도 강릉시 포남동 1169-0,성덕포남로168번안길 3,강원도 강릉시 포남동,1169,0,(1169-0),60.0215,15300,4,2021,2021-08-17,254.908658,1,연립다세대,Y,LH
163,강원도 강릉시 포남동 1169-0,성덕포남로168번안길 3,강원도 강릉시 포남동,1169,0,(1169-0),60.0215,15300,4,2021,2021-08-17,254.908658,1,연립다세대,Y,LH


In [24]:
prac_df['소유자'].value_counts()

LH    222
Name: 소유자, dtype: int64

In [25]:
%%time
prac_df.to_excel('./공공주택찾기/강원도/LH_거래사례_20180101_20220224.xlsx', index=False)

Wall time: 66 ms
